# IST718 Big Data Final Project - SF_crime
##### Project1 - Chiau Yin Yang, Qing Chen, Zilong Chen

We ran our models on databricks and download the python notebook for submission

In [2]:
# import package
from csv import reader
from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.ml import feature, regression, Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import functions as fn, Row
from pyspark.sql.types import *
from pyspark import sql
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import warnings

import os

## Convert Data to Spark DataFrame

In [4]:
# Load dataset
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.format("csv").option("header", "true").load("FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv")


In [5]:
# check files path
%fs ls FileStore/tables

path,name,size
dbfs:/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv,300601428
dbfs:/FileStore/tables/crime_type_predefined-1fa1e.csv,crime_type_predefined-1fa1e.csv,615


In [6]:
# view dataframe
df.show(5)
print(df.count())

+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+
IncidntNum| Category| Descript|DayOfWeek| Date| Time|PdDistrict|Resolution| Address| X| Y| Location| PdId|
+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+
 150060275| NON-CRIMINAL| LOST PROPERTY| Monday|01/19/2015|14:00| MISSION| NONE|18TH ST / VALENCI...| -122.42158168137|37.7617007179518|(37.7617007179518...|15006027571000|
 150098210| ROBBERY|ROBBERY, BODILY F...| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821003074|
 150098210| ASSAULT|AGGRAVATED ASSAUL...| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821004014|
 150098210|SECONDARY CODES| DOMESTIC VIOLENCE| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821015200|
 150098226| VANDALISM|MALICIOUS MISCHIE...| Tuesday|01/27/2015|19:00| NORTHERN| NONE|LOMBARD ST / LAGU...|-122.431118543788|37.8004687042875|(37.8004687042875...|15009822628160|
+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+
only showing top 5 rows

1453354

In [7]:
# data-processing: drop 1 NA
df1 = df.na.drop()
df1.count()

Out[ 12 ]: 1453353

In [8]:
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import udf
from datetime import datetime
# Convert Date type from string to unix_timestamp
func = udf (lambda x: datetime.strptime(x, '%m/%d/%Y'), DateType())
# Add 'Year', 'Month', 'Day' columns to the dataframe
df_with_timestamp = df1.withColumn('Spark_Date', func(df.Date))
df_with_timestamp = df_with_timestamp.withColumn('Year', year(df_with_timestamp.Spark_Date)).withColumn('Month', month(df_with_timestamp.Spark_Date))
df_with_timestamp = df_with_timestamp.withColumn('Day', dayofmonth(df_with_timestamp.Spark_Date))


In [9]:
df_with_timestamp.show(5)

+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+----------+----+-----+---+
IncidntNum| Category| Descript|DayOfWeek| Date| Time|PdDistrict|Resolution| Address| X| Y| Location| PdId|Spark_Date|Year|Month|Day|
+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+----------+----+-----+---+
 150060275| NON-CRIMINAL| LOST PROPERTY| Monday|01/19/2015|14:00| MISSION| NONE|18TH ST / VALENCI...| -122.42158168137|37.7617007179518|(37.7617007179518...|15006027571000|2015-01-19|2015| 1| 19|
 150098210| ROBBERY|ROBBERY, BODILY F...| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821003074|2015-02-01|2015| 2| 1|
 150098210| ASSAULT|AGGRAVATED ASSAUL...| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821004014|2015-02-01|2015| 2| 1|
 150098210|SECONDARY CODES| DOMESTIC VIOLENCE| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821015200|2015-02-01|2015| 2| 1|
 150098226| VANDALISM|MALICIOUS MISCHIE...| Tuesday|01/27/2015|19:00| NORTHERN| NONE|LOMBARD ST / LAGU...|-122.431118543788|37.8004687042875|(37.8004687042875...|15009822628160|2015-01-27|2015| 1| 27|
+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+----------+----+-----+---+
only showing top 5 rows

### Quick analysis for crime incidents by time 
- rank of crime by year

In [11]:
df_crimes_2003_to_2018_yearly = df_with_timestamp.groupBy('Year').count().orderBy('count', ascending=False)
df_crimes_2003_to_2018_yearly.cache()

Out[ 7 ]: DataFrame[Year: int, count: bigint]

In [12]:
df_crimes_2003_to_2018_yearly.show(10)

In [13]:
Crime_by_month = df_with_timestamp.groupBy('Month').count().orderBy('count', ascending=False)
Crime_by_month.show()

In [14]:
Crime_by_month2 = df_with_timestamp.groupBy('Month').count().orderBy('Month', ascending=True)
Crime_by_month2.show()

In [15]:
# Convert the Date and Time from String to Integer
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import year, month, dayofmonth, hour, minute
from datetime import datetime
# Convert Time type from string to timestamp
func_timestamp = udf (lambda x: datetime.strptime(x, '%H:%M'), TimestampType())
# Add 'Year', 'Month', 'Day' columns to the dataframe
df_with_timestamp = df_with_timestamp.withColumn('Spark_Time', func_timestamp(df_with_timestamp.Time))
df_with_timestamp = df_with_timestamp.withColumn('Hour', hour(df_with_timestamp.Spark_Time)).withColumn('Minute', minute(df_with_timestamp.Spark_Time))

In [16]:
show_timestamp = df_with_timestamp.select('IncidntNum', 'Category', 'Year', 'Month', 'Day', 'Hour', 'Minute')
show_timestamp.show(5)

In [17]:
show_timestamp2=df_with_timestamp.select('Category', 'PdDistrict', 'Year', 'Month', 'Day', 'Hour','Resolution')
show_timestamp2.show(10)

In [18]:
# covert target attribute - resolution to numeric
df_with_timestamp = df_with_timestamp.withColumn(
    'Res_num', fn.when((fn.col('Resolution') == 'NONE') | (fn.col('Resolution') == 'UNFOUNDED'), 0).otherwise(1))

In [19]:
df_with_timestamp.groupby('Res_num').count().show()

+-------+------+
Res_num| count|
+-------+------+
 1|515964|
 0|937389|
+-------+------+

## Feature Engineering
##### Create labeled dummy variables for categorical columns
- Location (PD District)
- Day of week
- Crime category (broken into 5 levels by the degree of the crime, defined by us)
- Incident hour (broken into 5 levels to even out the number of incidents)
- Month as 12 levels

In [20]:
# location
df_with_timestamp = df_with_timestamp.withColumn('SOUTHERN',(fn.col('PdDistrict')== 'SOUTHERN').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('MISSION',(fn.col('PdDistrict')== 'MISSION').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('NORTHERN',(fn.col('PdDistrict')== 'NORTHERN').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('CENTRAL',(fn.col('PdDistrict')== 'CENTRAL').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('BAYVIEW',(fn.col('PdDistrict')== 'BAYVIEW').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('TENDERLOIN',(fn.col('PdDistrict')== 'TENDERLOIN').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('INGLESIDE',(fn.col('PdDistrict')== 'INGLESIDE').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('TARAVAL',(fn.col('PdDistrict')== 'TARAVAL').cast('int'))
df_with_timestamp = df_with_timestamp.withColumn('PARK',(fn.col('PdDistrict')== 'PARK').cast('int'))
#df_with_timestamp = df_with_timestamp.withColumn('RICHMOND',(fn.col('PdDistrict')== 'RICHMOND').cast('int'))
# use Richmond as reference for inference

In [21]:
df_with_timestamp.show(5)

+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+----------+----+-----+---+-------------------+----+------+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+
IncidntNum| Category| Descript|DayOfWeek| Date| Time|PdDistrict|Resolution| Address| X| Y| Location| PdId|Spark_Date|Year|Month|Day| Spark_Time|Hour|Minute|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|
+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+----------+----+-----+---+-------------------+----+------+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+
 150060275| NON-CRIMINAL| LOST PROPERTY| Monday|01/19/2015|14:00| MISSION| NONE|18TH ST / VALENCI...| -122.42158168137|37.7617007179518|(37.7617007179518...|15006027571000|2015-01-19|2015| 1| 19|1900-01-01 14:00:00| 14| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0|
 150098210| ROBBERY|ROBBERY, BODILY F...| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821003074|2015-02-01|2015| 2| 1|1900-01-01 15:45:00| 15| 45| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0|
 150098210| ASSAULT|AGGRAVATED ASSAUL...| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821004014|2015-02-01|2015| 2| 1|1900-01-01 15:45:00| 15| 45| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0|
 150098210|SECONDARY CODES| DOMESTIC VIOLENCE| Sunday|02/01/2015|15:45|TENDERLOIN| NONE|300 Block of LEAV...|-122.414406029855|37.7841907151119|(37.7841907151119...|15009821015200|2015-02-01|2015| 2| 1|1900-01-01 15:45:00| 15| 45| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0|
 150098226| VANDALISM|MALICIOUS MISCHIE...| Tuesday|01/27/2015|19:00| NORTHERN| NONE|LOMBARD ST / LAGU...|-122.431118543788|37.8004687042875|(37.8004687042875...|15009822628160|2015-01-27|2015| 1| 27|1900-01-01 19:00:00| 19| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
+----------+---------------+--------------------+---------+----------+-----+----------+----------+--------------------+-----------------+----------------+--------------------+--------------+----------+----+-----+---+-------------------+----+------+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+
only showing top 5 rows

In [22]:
# Load the pre-defined crime level dataset
df_crime_type = spark.read.format("csv").option("header", "true").load("FileStore/tables/crime_type_predefined-1fa1e.csv")

In [23]:
#df_crime_type.drop(index=0, inplace = True)
df_crime_type.head(3)

Out[ 20 ]: 
[Row(crime_type='NON-CRIMINAL', level='1'),
 Row(crime_type='WARRANTS', level='1'),
 Row(crime_type='GAMBLING', level='1')]

In [24]:
import pyspark.sql
from pyspark.sql import functions as fn


In [25]:
#df_with_timestamp.printSchema()

In [26]:
# only save necessary columns to save some spaces
df_final = df_with_timestamp.select('Category', 'Year', 'Month', 'Hour','Res_num',
                            'SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW',
                            'TENDERLOIN','INGLESIDE','TARAVAL','PARK','DayOfWeek')
df_final.show(5)

+---------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+
 Category|Year|Month|Hour|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|DayOfWeek|
+---------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+
 NON-CRIMINAL|2015| 1| 14| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| Monday|
 ROBBERY|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday|
 ASSAULT|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday|
SECONDARY CODES|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday|
 VANDALISM|2015| 1| 19| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| Tuesday|
+---------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+
only showing top 5 rows

In [27]:
#create dummy variable for day of the week - use wednesday as our reference 
df_final = df_final.withColumn('Sunday', fn.when(fn.col('DayOfWeek') == 'Sunday', 1).otherwise(0))
df_final = df_final.withColumn('Monday', fn.when(fn.col('DayOfWeek') == 'Monday', 1).otherwise(0))
df_final = df_final.withColumn('Tuesday', fn.when(fn.col('DayOfWeek') == 'Tuesday', 1).otherwise(0))
df_final = df_final.withColumn('Thursday', fn.when(fn.col('DayOfWeek') == 'Thursday', 1).otherwise(0))
df_final = df_final.withColumn('Friday', fn.when(fn.col('DayOfWeek') == 'Friday', 1).otherwise(0))
df_final = df_final.withColumn('Saturday', fn.when(fn.col('DayOfWeek') == 'Saturday', 1).otherwise(0))

df_final.show(3)

+------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+
 Category|Year|Month|Hour|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|DayOfWeek|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|
+------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+
NON-CRIMINAL|2015| 1| 14| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| Monday| 0| 1| 0| 0| 0| 0|
 ROBBERY|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday| 1| 0| 0| 0| 0| 0|
 ASSAULT|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday| 1| 0| 0| 0| 0| 0|
+------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+
only showing top 3 rows

In [28]:
# morning as reference
df_final = df_final.withColumn('Morning', fn.when((fn.col('Hour') == 1) | (fn.col('Hour') == 2) |
                                                  (fn.col('Hour') == 3) |(fn.col('Hour') == 4) |
                                                  (fn.col('Hour') == 5) |(fn.col('Hour') == 6) |
                                                  (fn.col('Hour') == 7) | (fn.col('Hour') == 8), 1).otherwise(0))


In [29]:
#create dummy variable for day of the week - use wednesday as our reference
df_final = df_final.withColumn('Near noon', fn.when((fn.col('Hour') == 9) | (fn.col('Hour') == 10) |
                                              (fn.col('Hour') == 11) | (fn.col('Hour') == 12), 1).otherwise(0))
df_final = df_final.withColumn('Afternoon', fn.when((fn.col('Hour') == 13) | (fn.col('Hour') == 14) |
                                            (fn.col('Hour') == 15) | (fn.col('Hour') == 16) |
                                             (fn.col('Hour') == 17), 1).otherwise(0))
df_final = df_final.withColumn('evening', fn.when((fn.col('Hour') >= 18) & (fn.col('Hour') <= 20), 1).otherwise(0))
df_final = df_final.withColumn('Night', fn.when((fn.col('Hour') == 21) | (fn.col('Hour') == 22) |
                                                 (fn.col('Hour') == 23) | (fn.col('Hour') == 0), 1).otherwise(0))

df_final.show(3)
#training_df = df_final.where('Year >= 2008 and Year < 2014')                       

+------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+---------+---------+-------+-----+
 Category|Year|Month|Hour|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|DayOfWeek|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|
+------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+---------+---------+-------+-----+
NON-CRIMINAL|2015| 1| 14| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| Monday| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0|
 ROBBERY|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0|
 ASSAULT|2015| 2| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0|
+------------+----+-----+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+---------+---------+-------+-----+
only showing top 3 rows

In [30]:
# dropping morning cuz it is baseline reference
df_final = df_final.drop('Morning')
df_final.show(2)

+------------+----+-----+---+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+---------+---------+-------+-----+
 Category|Year|Month|Day|Hour|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|DayOfWeek|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|
+------------+----+-----+---+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+---------+---------+-------+-----+
NON-CRIMINAL|2015| 1| 19| 14| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| Monday| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0|
 ROBBERY|2015| 2| 1| 15| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| Sunday| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0|
+------------+----+-----+---+----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+---------+------+------+-------+--------+------+--------+---------+---------+-------+-----+
only showing top 2 rows

In [31]:
# dropping columns that we dont need for modeling
df_final = df_final.drop('Hour')
df_final = df_final.drop('DayOfWeek')
df_final.show(2)

+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+
 Category|Year|Month|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|
+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+
NON-CRIMINAL|2015| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0|
 ROBBERY|2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0|
+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+
only showing top 2 rows

In [32]:
# checking the size of the dataframe
print((df_final.count(), len(df_final.columns)))

(1453353, 24)

In [33]:
df_crime_type = df_crime_type.selectExpr("crime_type as Category", "level as Level")
df_crime_type.show(2)

+------------+-----+
 Category|Level|
+------------+-----+
NON-CRIMINAL| 1|
 WARRANTS| 1|
+------------+-----+
only showing top 2 rows

In [34]:

crime_join = df_final.join(df_crime_type, on='Category',how='left')
# Could also use 'right_outer'
crime_join.show(3)


+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+
 Category|Year|Month|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|Level|
+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+
NON-CRIMINAL|2015| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 1|
 ROBBERY|2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 4|
 ASSAULT|2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 4|
+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+
only showing top 3 rows

In [35]:
#df_final = df_final.withColumn('Very Minor', fn.when(fn.col('Level') == '1', 1).otherwise(0)) 
# as our reference 
crime_join = crime_join.withColumn('Minor', fn.when(fn.col('Level') == '2', 1).otherwise(0))
crime_join = crime_join.withColumn('Medium', fn.when(fn.col('Level') == '3', 1).otherwise(0))
crime_join = crime_join.withColumn('Servere', fn.when(fn.col('Level') == '4', 1).otherwise(0))
crime_join = crime_join.withColumn('Very Servere', fn.when(fn.col('Level') == '5', 1).otherwise(0))

crime_join.show(3)

+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+-----+------+-------+------------+
 Category|Year|Month|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|Level|Minor|Medium|Servere|Very Servere|
+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+-----+------+-------+------------+
NON-CRIMINAL|2015| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0|
 ROBBERY|2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 4| 0| 0| 1| 0|
 ASSAULT|2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 4| 0| 0| 1| 0|
+------------+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+-----+------+-------+------------+
only showing top 3 rows

In [36]:
crime_join.groupby('Level').count().show()
# why is there missing values?? yay it is all fixed!!

+-----+------+
Level| count|
+-----+------+
 3|489779|
 5|118431|
 1|249718|
 4|301876|
 2|293549|
+-----+------+

In [37]:
df_final = crime_join.drop('Level')
df_final = df_final.drop('Category')
df_final.show()

+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+------+-------+------------+
Year|Month|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|Minor|Medium|Servere|Very Servere|
+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+------+-------+------------+
2015| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1|
2015| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
2015| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 1|
2015| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 1| 0|
2015| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 0| 1| 0|
2015| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 1| 0| 0| 0| 1| 0| 0|
2015| 2| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0|
2015| 2| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1|
2015| 2| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1|
2015| 2| 1| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
2015| 2| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0|
2015| 2| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0| 0|
2015| 2| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0|
2015| 2| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0|
2015| 2| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0|
+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+------+-------+------------+
only showing top 20 rows

In [38]:
# create month category - dec as reference
df_final = df_final.withColumn('Jan', fn.when(fn.col('Month') == '1', 1).otherwise(0))
df_final = df_final.withColumn('Feb', fn.when(fn.col('Month') == '2', 1).otherwise(0))
df_final = df_final.withColumn('Mar', fn.when(fn.col('Month') == '3', 1).otherwise(0))
df_final = df_final.withColumn('Apr', fn.when(fn.col('Month') == '4', 1).otherwise(0))
df_final = df_final.withColumn('May', fn.when(fn.col('Month') == '5', 1).otherwise(0))
df_final = df_final.withColumn('Jun', fn.when(fn.col('Month') == '6', 1).otherwise(0))
df_final = df_final.withColumn('Jul', fn.when(fn.col('Month') == '7', 1).otherwise(0))
df_final = df_final.withColumn('Aug', fn.when(fn.col('Month') == '8', 1).otherwise(0))
df_final = df_final.withColumn('Sep', fn.when(fn.col('Month') == '9', 1).otherwise(0))
df_final = df_final.withColumn('Oct', fn.when(fn.col('Month') == '10', 1).otherwise(0))
df_final = df_final.withColumn('Nov', fn.when(fn.col('Month') == '11', 1).otherwise(0))
df_final.show(3)

+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+------+-------+------------+---+---+---+---+---+---+---+---+---+---+---+
Year|Month|Res_num|SOUTHERN|MISSION|NORTHERN|CENTRAL|BAYVIEW|TENDERLOIN|INGLESIDE|TARAVAL|PARK|Sunday|Monday|Tuesday|Thursday|Friday|Saturday|Near noon|Afternoon|evening|Night|Minor|Medium|Servere|Very Servere|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|
+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+------+-------+------------+---+---+---+---+---+---+---+---+---+---+---+
2015| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
2015| 2| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 1| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+----+-----+-------+--------+-------+--------+-------+-------+----------+---------+-------+----+------+------+-------+--------+------+--------+---------+---------+-------+-----+-----+------+-------+------------+---+---+---+---+---+---+---+---+---+---+---+
only showing top 3 rows

In [39]:
# create year category - 2008 as reference
df_final = df_final.withColumn('Year_09', fn.when(fn.col('Year') == '2009', 1).otherwise(0))
df_final = df_final.withColumn('Year_10', fn.when(fn.col('Year') == '2010', 1).otherwise(0))
df_final = df_final.withColumn('Year_11', fn.when(fn.col('Year') == '2011', 1).otherwise(0))
df_final = df_final.withColumn('Year_12', fn.when(fn.col('Year') == '2012', 1).otherwise(0))
df_final = df_final.withColumn('Year_13', fn.when(fn.col('Year') == '2013', 1).otherwise(0))
df_final = df_final.withColumn('Year_14', fn.when(fn.col('Month') == '6', 1).otherwise(0))
df_final = df_final.withColumn('Year_15', fn.when(fn.col('Month') == '7', 1).otherwise(0))
df_final = df_final.withColumn('Year_16', fn.when(fn.col('Month') == '8', 1).otherwise(0))
df_final = df_final.withColumn('Year_17', fn.when(fn.col('Month') == '9', 1).otherwise(0))
df_final = df_final.withColumn('Month', fn.when(fn.col('Month') == '10', 1).otherwise(0))
df_final = df_final.withColumn('Month', fn.when(fn.col('Month') == '11', 1).otherwise(0))
df_final.show(3)

## Split dataset into training, validation and testing by the year of the incidents
- Follow 0.6, 0.3, and 0.1 rule to split (6 years of data as training etc)

In [40]:
# or split by year
training_df = df_final.where('Year >= 2008 and Year < 2014')
validation_df = df_final.where('Year >= 2014 and Year < 2017')
testing_df = df_final.where('Year >= 2017')

In [41]:
print("# points in training: ", training_df.count())
print("# points in validation: ", validation_df.count())
print("# points in testing: ", testing_df.count())

# points in training: 841044
# points in validation: 457536
# points in testing: 154773

In [42]:
testing_df.groupby('Res_num').count().show()
# points in training: 841044 # points in validation: 457536 # points in testing: 154773

+-------+------+
Res_num| count|
+-------+------+
 1| 37031|
 0|117742|
+-------+------+

# Modeling - run with different variables
- Random Forest (refere as rf)
- Logistic regression (refer as lr)
- Gradient boosting (refer as gb)

model# differentiates the number of variables

#### Variables that are considered:
- location
- Crime category
- Incident time
    - Hour (5 levels)
    - Month (12 levels)
    - Day of the week (7 levels)
    
Parameters used in model if any:
1. (RF) number of trees (10, 20, 30, 50, 80, 100)
2. (RF) cachenodeID
3. (GB, RF) seed = 0 for same result
4. (GB, LR) maxIter=10
5. (LR) regParam, elasticNetParam

In [43]:
# Functionality for computing features
from pyspark.ml import feature
# Functionality for regression
from pyspark.ml import regression
# Funcionality for classification
from pyspark.ml import classification
# Object for creating sequences of transformations
from pyspark.ml import Pipeline, evaluation

In [44]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics

In [45]:
from pyspark.sql.functions import isnan
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import StringIndexer

In [46]:
# category only
crime_m1 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium',
                                                               'Servere', 'Very Servere'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10)])
crime_m1_fitted = crime_m1.fit(training_df)
transformer_m1 = crime_m1_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
lr_AUC1 = evaluator.evaluate(transformer_m1)
#regParam=0, elasticNetParam=0
print('lr_m1 = ',lr_AUC1)

lr_m1 = 0.752094343928631

In [47]:
# category only
gb_m1 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium',
                                                               'Servere', 'Very Servere'],
                                        outputCol='features'),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='features', maxIter=10)])
gb_m1_fitted = gb_m1.fit(training_df)
gb_transformer_m1 = gb_m1_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
gb_AUC1 = evaluator.evaluate(gb_transformer_m1)
#regParam=0, elasticNetParam=0
print('gb_m1 = ',gb_AUC1)

gb_m1 = 0.7520943439286308

In [48]:
# category only
rf_m1 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium',
                                                               'Servere', 'Very Servere'],
                                        outputCol='features'),
                classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m1_fitted = rf_m1.fit(training_df)
rf_m1_trans = rf_m1_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC1 = evaluator.evaluate(rf_m1_trans)
print('rf_m1 = ',rf_AUC1)

rf_m1 = 0.752094343928631

In [49]:
# crime type + location
crime_m2 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10)])
crime_m2_fitted = crime_m2.fit(training_df)
transformer_m2 = crime_m2_fitted.transform(validation_df)
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="Res_num")
print('lr_AUC2 = ', evaluator.evaluate(transformer_m2))

lr_AUC2 = 0.7636711022971836

In [50]:
# crime type + location
gb_m2 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='features', maxIter=10)])
gb_m2_fitted = gb_m2.fit(training_df)
gb_transformer_m2 = gb_m2_fitted.transform(validation_df)
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="Res_num")
print('gb_AUC2 = ', evaluator.evaluate(gb_transformer_m2))

gb_AUC2 = 0.7629948863156344

In [51]:
rf_m2 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m2_fitted = rf_m2.fit(training_df)
rf_m2_trans = rf_m2_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC2 = evaluator.evaluate(rf_m2_trans)
print('rf_m2 = ',rf_AUC2)

rf_m2 = 0.7576967542235056

In [52]:
# crime level + year + month + location + hour level --> updated: removed year
crime_m9 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features')])
crime_m9_fitted = crime_m9.fit(training_df)
# removed day of the week
transformer_m9 = crime_m9_fitted.transform(validation_df)

print('AUC9 = ', evaluator.evaluate(transformer_m9))

AUC9 = 0.7649703312563588

In [53]:

# crime level + year + month + location + hour level
gb_m9 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere', 'Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='features',maxIter=10)])
gb_m9_fitted = gb_m9.fit(training_df)
# removed day of the week
gb_transformer_m9 = gb_m9_fitted.transform(validation_df)

print('AUC9 = ', evaluator.evaluate(gb_transformer_m9))

AUC9 = 0.7665906305842367

In [54]:
# crime level + year + month + location + hour level
rf_m9 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
         classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m9_fitted = rf_m9.fit(training_df)
rf_m9_trans = rf_m9_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC9 = evaluator.evaluate(rf_m9_trans)
print('rf_m9 = ',rf_AUC9)

rf_m9 = 0.7439949632659307

In [55]:
# crime level + year + month + day + hour
crime_m3 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10)])
crime_m3_fitted = crime_m3.fit(training_df)
transformer_m3 = crime_m3_fitted.transform(validation_df)
print('lr_AUC3 = ', evaluator.evaluate(transformer_m3))

lr_AUC3 = 0.7567503055955304

In [56]:
#GBTClassifier
# crime level + year + month + day + hour
gb_m3 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night'],
                                        outputCol='features'),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='features', maxIter=10)])
gb_m3_fitted = gb_m3.fit(training_df)
gb_transformer_m3 = gb_m3_fitted.transform(validation_df)
print('lr_AUC3 = ', evaluator.evaluate(gb_transformer_m3))

lr_AUC3 = 0.760084956337706

In [57]:
# crime level + year + month + day + hour
rf_m3 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night'],
                                        outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m3_fitted = rf_m3.fit(training_df)
rf_m3_trans = rf_m3_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC3 = evaluator.evaluate(rf_m3_trans)
print('rf_m3 = ',rf_AUC3)

rf_m3 = 0.7544324712555827

In [58]:
# crime level + day, year month, hour, location = all
crime_m4 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10)])
crime_m4_fitted = crime_m4.fit(training_df)
transformer_m4 = crime_m4_fitted.transform(validation_df)
print('lr_AUC4 = ', evaluator.evaluate(transformer_m4))

lr_AUC4 = 0.7645646655730021

In [59]:

# crime level + day, year month, hour, location = all
gb_m4 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='features', maxIter=10)])
gb_m4_fitted = gb_m4.fit(training_df)
gb_transformer_m4 = gb_m4_fitted.transform(validation_df)
print('lr_AUC4 = ', evaluator.evaluate(gb_transformer_m4))

lr_AUC4 = 0.7665909570024958

In [60]:
# crime level + day, year month, hour, location = all
rf_m4 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m4_fitted = rf_m4.fit(training_df)
rf_m4_trans = rf_m4_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC4 = evaluator.evaluate(rf_m4_trans)
print('rf_m4 = ',rf_AUC4)

rf_m4 = 0.7561433260785806

In [61]:
rf_m21 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=50, cacheNodeIds = True)])
rf_m21_fitted = rf_m21.fit(training_df)
rf_m21_trans = rf_m21_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC21 = evaluator.evaluate(rf_m21_trans)
print('rf_m21 = ',rf_AUC21)

rf_m21 = 0.7610003610178669

In [62]:
rf_m22 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=30, cacheNodeIds = True)])
rf_m22_fitted = rf_m22.fit(training_df)
rf_m22_trans = rf_m22_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC22 = evaluator.evaluate(rf_m22_trans)
print('rf_m22 = ',rf_AUC22)

rf_m22 = 0.7637457819693152

In [63]:
rf_m23 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=40, cacheNodeIds = True)])
rf_m23_fitted = rf_m23.fit(training_df)
rf_m23_trans = rf_m23_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC23 = evaluator.evaluate(rf_m23_trans)
print('rf_m23 = ',rf_AUC23)

rf_m23 = 0.7621921476810491

In [64]:
rf_m24 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=80, cacheNodeIds = True)])
rf_m24_fitted = rf_m24.fit(training_df)
rf_m24_trans = rf_m24_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC24 = evaluator.evaluate(rf_m24_trans)
print('rf_m24 = ',rf_AUC24)

rf_m24 = 0.7635109835212445

In [65]:
rf_m25 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','SOUTHERN','MISSION','NORTHERN','CENTRAL','BAYVIEW','TENDERLOIN',
                    'INGLESIDE','TARAVAL','PARK'], outputCol='features'),
            classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=100, cacheNodeIds = True)])
rf_m25_fitted = rf_m25.fit(training_df)
rf_m25_trans = rf_m25_fitted.transform(validation_df)
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='Res_num')
rf_AUC25 = evaluator.evaluate(rf_m25_trans)
print('rf_m25 = ',rf_AUC25)

rf_m25 = 0.7634092093029908

In [66]:
# just pick the highest AUC 
# crime level + year + month + location + hour level
crime_m9_1 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.3, elasticNetParam=0.8)])
crime_m9_1_fitted = crime_m9_1.fit(training_df)


In [67]:
# crime level + day, year month, hour, location = all
crime_m4_1 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.3, elasticNetParam=0.8)])
crime_m4_1_fitted = crime_m4_1.fit(training_df)

In [68]:
AUC9_1 = evaluator.evaluate(crime_m9_1_fitted.transform(validation_df))
AUC4_1 = evaluator.evaluate(crime_m4_1_fitted.transform(validation_df))

print("Model 4 - ALL + parameters AUC: ", AUC4_1)
print("Model 9_1 - category + time + parameters AUC: ", AUC9_1)
print("Model 4 - ALL AUC: ", evaluator.evaluate(transformer_m4))
print("Model 9 - category + location + time AUC: ", evaluator.evaluate(transformer_m9))


Model 4 - ALL + parameters AUC: 0.5
Model 9_1 - category + time + parameters AUC: 0.5
Model 4 - ALL AUC: 0.7645646655730022
Model 9 - category + location + time AUC: 0.7646654151462846

In [69]:
# crime level + day, year month, hour, location = all
crime_m4_2 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.1, elasticNetParam=0.4)])
crime_m4_2_fitted = crime_m4_2.fit(training_df)

In [70]:
# just pick the highest AUC 
# crime level + year + month + location + hour level
crime_m9_2 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.1, elasticNetParam=0.4)])
crime_m9_2_fitted = crime_m9_2.fit(training_df)


In [71]:
AUC9_2 = evaluator.evaluate(crime_m9_2_fitted.transform(validation_df))
AUC4_2 = evaluator.evaluate(crime_m4_2_fitted.transform(validation_df))

print("Model 4_2 - ALL + parameters AUC: ", AUC4_2)
print("Model 9_2 - category + time + parameters AUC: ", AUC9_2)

Model 4_2 - ALL + parameters AUC: 0.7488183414900433
Model 9_2 - category + time + parameters AUC: 0.7488183414900434

In [72]:
# crime level + day, year month, hour, location = all
crime_m4_3 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.02, elasticNetParam=0.2)])
crime_m4_3_fitted = crime_m4_3.fit(training_df)

In [73]:
# just pick the highest AUC 
# crime level + year + month + location + hour level
crime_m9_3 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.02, elasticNetParam=0.2)])
crime_m9_3_fitted = crime_m9_3.fit(training_df)


In [74]:
AUC9_3 = evaluator.evaluate(crime_m9_3_fitted.transform(validation_df))
AUC4_3 = evaluator.evaluate(crime_m4_3_fitted.transform(validation_df))

print("Model 4_3 - ALL + parameters AUC: ", AUC4_3)
print("Model 9_3 - category + time + parameters AUC: ", AUC9_3)

Model 4_3 - ALL + parameters AUC: 0.7647424100366792
Model 9_3 - category + time + parameters AUC: 0.7642961044272933

In [75]:
# crime level + day, year month, hour, location = all
crime_m4_4 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.01, elasticNetParam=0.1)])
crime_m4_4_fitted = crime_m4_4.fit(training_df)

In [76]:
# just pick the highest AUC 
# crime level + year + month + location + hour level
crime_m9_4 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='features', maxIter=10,
                                                  regParam=0.01, elasticNetParam=0.1)])
crime_m9_4_fitted = crime_m9_4.fit(training_df)


In [77]:
AUC9_4 = evaluator.evaluate(crime_m9_4_fitted.transform(validation_df))
AUC4_4 = evaluator.evaluate(crime_m4_4_fitted.transform(validation_df))

print("Model 4_4 - ALL + parameters AUC: ", AUC4_4)
print("Model 9_4 - category + time + parameters AUC: ", AUC9_4)

Model 4_4 - ALL + parameters AUC: 0.7644687919435298
Model 9_4 - category + time + parameters AUC: 0.7640214452477342

## Evaluate the best models on testing df
- one for each model

In [78]:
# L1 = 0.02, L3 = 0.2 - m4 (all variable + parameter)
lr_AUC_best = evaluator.evaluate(crime_m9_fitted.transform(testing_df))
lr_AUC_best

Out[ 158 ]: 0.7686324149521971

In [79]:
# all variables except day of the week without parameters
rf_AUC_best = evaluator.evaluate(rf_m2_fitted.transform(testing_df))
rf_AUC_best

Out[ 157 ]: 0.7641741337985635

In [80]:
# all variables except day of the week without parameters
gb_AUC_best = evaluator.evaluate(gb_m9_fitted.transform(testing_df))
gb_AUC_best

Out[ 156 ]: 0.7717859729851462

In [81]:
crime_m9_fitted.transform(testing_df).select('Res_num', 'prediction').show(10)

+-------+----------+
Res_num|prediction|
+-------+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+-------+----------+
only showing top 10 rows

In [82]:
crime_m4_3_fitted.transform(testing_df).select('Res_num', 'prediction').show(10)

+-------+----------+
Res_num|prediction|
+-------+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+-------+----------+
only showing top 10 rows

In [83]:
crime_m9_fitted.transform(testing_df).select(fn.count(fn.col('prediction') == fn.col('Res_num')) / (fn.count(fn.expr('prediction = 1')))).show()


+---------------------------------------------------------+
(count((prediction = Res_num)) / count((prediction = 1)))|
+---------------------------------------------------------+
 1.0|
+---------------------------------------------------------+

# Evaluate the performance by confusion matrixs
- calculate precision and recall for each model

In [84]:
lr_model_9 = crime_m9_fitted.transform(testing_df)
#.select((fn.col('prediction') == fn.col('Res_num'))).show()

TP = int(lr_model_9.filter((lr_model_9["Res_num"] == 1) & (lr_model_9["prediction"] == 1)).count())


In [85]:
TP_FN = int(lr_model_9.filter(lr_model_9["Res_num"] == 1).count())
TP_FP = int(lr_model_9.filter(lr_model_9["prediction"] == 1).count())

In [86]:
lr_recall = int(TP) / int(TP_FN)
lr_precision = int(TP) / int(TP_FP)

In [87]:
gb_model_9 = gb_m9_fitted.transform(testing_df)
#.select((fn.col('prediction') == fn.col('Res_num'))).show()
gb_TP = int(gb_model_9.filter((gb_model_9["Res_num"] == 1) & (gb_model_9["prediction"] == 1)).count())
gb_TP_FN = int(gb_model_9.filter(gb_model_9["Res_num"] == 1).count())
gb_TP_FP = int(gb_model_9.filter(gb_model_9["prediction"] == 1).count())

In [88]:
gb_recall = int(gb_TP) / int(gb_TP_FN)
gb_precision = int(gb_TP) / int(gb_TP_FP)

In [89]:
rf_model_2 = rf_m2_fitted.transform(testing_df)
#.select((fn.col('prediction') == fn.col('Res_num'))).show()
rf_TP = int(rf_model_2.filter((rf_model_2["Res_num"] == 1) & (rf_model_2["prediction"] == 1)).count())
rf_TP_FN = int(rf_model_2.filter(rf_model_2["Res_num"] == 1).count())
rf_TP_FP = int(rf_model_2.filter(rf_model_2["prediction"] == 1).count())

In [90]:
rf_recall = int(rf_TP) / int(rf_TP_FN)
rf_precision = int(rf_TP) / int(rf_TP_FP)

In [91]:
print('RF',rf_recall)
print('RF-P',rf_precision)
print('LR',lr_recall)
print('LR-P',lr_precision)
print('GB',gb_recall)
print('GB-P',gb_precision)


RF 0.49056196159974075
RF-P 0.5185396626038307
LR 0.5432745537522616
LR-P 0.4912700544553247
GB 0.5262887850719667
GB-P 0.5044650946082365

In [92]:
testing_df.count()

Out[ 63 ]: 154773

In [93]:
# calculating recall, the higher the recall, the lower the false negative is

crime_m9_fitted.transform(testing_df).select(fn.sum('prediction = 1'))

Out[ 54 ]: DataFrame[Year: int, Month: int, Day: int, Res_num: int, SOUTHERN: int, MISSION: int, NORTHERN: int, CENTRAL: int, BAYVIEW: int, TENDERLOIN: int, INGLESIDE: int, TARAVAL: int, PARK: int, Sunday: int, Monday: int, Tuesday: int, Thursday: int, Friday: int, Saturday: int, Near noon: int, Afternoon: int, evening: int, Night: int, Minor: int, Medium: int, Servere: int, Very Servere: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [94]:
crime_m9_fitted.transform(testing_df).select(fn.avg('prediction')).show()


+------------------+
 avg(prediction)|
+------------------+
0.2722309446738126|
+------------------+

In [95]:
crime_m9_fitted.stages[-1].coefficients

Out[ 75 ]: DenseVector([0.5714, -1.4023, -0.8335, 1.2617, 0.0, -0.0103, 0.4389, 0.5892, 0.1894, -0.0141, 0.4621, 1.2776, 0.2503, 0.0873, 0.3964, -0.2521, -0.0001, -0.1078, -0.151])

In [96]:
crime_m9_fitted.stages[-1].intercept

Out[ 76 ]: -0.3795459617913322

In [97]:
crime_m4_fitted.stages[-1].coefficients

Out[ 83 ]: DenseVector([0.27, -0.3321, -0.1588, 0.4346, 0.0002, -0.0038, -0.0077, 0.0045, 0.0129, 0.0049, -0.0286, -0.0131, -0.028, 0.0427, -0.0318, -0.0218, -0.0024, 0.068, -0.0856, -0.1353, 0.0226, 0.3068, -0.0408, -0.0928, -0.0278])

In [98]:
crime_m4_3_fitted.stages[-1].intercept

Out[ 77 ]: -0.3798297190777381

In [99]:
crime_m4_3_fitted.stages[-1].coefficients

Out[ 78 ]: DenseVector([0.6332, -1.1305, -0.5377, 1.3286, 0.0001, -0.0059, 0.0, 0.0, 0.0027, 0.0, -0.0383, -0.0004, -0.0492, 0.0816, -0.0465, -0.026, 0.0, 0.1957, -0.1619, -0.2749, 0.022, 0.7229, -0.059, -0.1746, -0.0224])

### Check if standard scalar and other paramters will change the results

In [100]:
crime_m4_5 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features', withMean = True, withStd = True),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='scalar_features', maxIter=10,
                                                  regParam=0.02, elasticNetParam=0.2)])
crime_m4_5_fitted = crime_m4_5.fit(training_df)

In [101]:
#GBTClassifier
gb_m4_5 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features', withMean = True, withStd = True),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='scalar_features', maxIter=10)])
gb_m4_5_fitted = gb_m4_5.fit(training_df)

In [102]:
rf_m4_5 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features', withMean = True, withStd = True),
                 classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m4_5_fitted = rf_m4_5.fit(training_df)

In [103]:
# crime level + year + month + location + hour level
crime_m9_5 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features',withMean = True, withStd = True),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='scalar_features', maxIter=10,
                                                  regParam=0.02, elasticNetParam=0.2)])
crime_m9_5_fitted = crime_m9_5.fit(training_df)

In [104]:
# crime level + year + month + location + hour level
gb_m9_5 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features',withMean = True, withStd = True),
                 classification.GBTClassifier(labelCol='Res_num', featuresCol='scalar_features', maxIter=10)])
gb_m9_5_fitted = gb_m9_5.fit(training_df)

In [105]:
# crime level + year + month + location + hour level
rf_m9_5 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features',withMean = True, withStd = True),
                 classification.RandomForestClassifier(labelCol='Res_num', featuresCol='features', seed = 0,
                            numTrees=10, cacheNodeIds = True)])
rf_m9_5_fitted = rf_m9_5.fit(training_df)

In [106]:
AUC9_5 = evaluator.evaluate(crime_m9_5_fitted.transform(validation_df))
AUC4_5 = evaluator.evaluate(crime_m4_5_fitted.transform(validation_df))

print("Model 4_5 - ALL + parameters AUC: ", AUC4_5)
print("Model 9_5 - category + time + parameters AUC: ", AUC9_5)

Model 4_5 - ALL + parameters AUC: 0.76431467359888
Model 9_5 - category + time + parameters AUC: 0.7643229693504273

In [107]:
gb_AUC9_5 = evaluator.evaluate(gb_m9_5_fitted.transform(validation_df))
gb_AUC4_5 = evaluator.evaluate(gb_m4_5_fitted.transform(validation_df))

print("Model 4_5 - ALL + parameters AUC: ", gb_AUC4_5)
print("Model 9_5 - category + time + parameters AUC: ", gb_AUC9_5)

Model 4_5 - ALL + parameters AUC: 0.7665909570024959
Model 9_5 - category + time + parameters AUC: 0.7663867773102793

In [108]:
rf_AUC9_5 = evaluator.evaluate(rf_m9_5_fitted.transform(validation_df))
rf_AUC4_5 = evaluator.evaluate(rf_m4_5_fitted.transform(validation_df))

print("rf_Model 4_5 - ALL AUC: ", rf_AUC4_5)
print("rf_Model 9_5 - category + time AUC: ", rf_AUC9_5)

rf_Model 4_5 - ALL AUC: 0.7561433260785807
rf_Model 9_5 - category + time AUC: 0.7439949632659305

In [109]:
df_final.select('Res_num').distinct().show()

+-------+
Res_num|
+-------+
 1|
 0|
+-------+

In [110]:
crime_m4_6 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','Sunday','Monday','Tuesday','Thursday','Friday','Saturday',
                    'Near noon','Afternoon','evening','Night','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK'],
                                        outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features', withMean = True, withStd = True),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='scalar_features', maxIter=10)])
crime_m4_6_fitted = crime_m4_6.fit(training_df)

In [111]:
# crime level + year + month + location + hour level
crime_m9_6 = Pipeline(stages=[feature.VectorAssembler(inputCols=['Minor', 'Medium', 'Servere',
                    'Very Servere','Year','Month','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night'], outputCol='features'),
                 feature.StandardScaler(inputCol='features', outputCol= 'scalar_features',withMean = True, withStd = True),
                 classification.LogisticRegression(labelCol='Res_num', featuresCol='scalar_features', maxIter=10)])
crime_m9_6_fitted = crime_m9_6.fit(training_df)

In [112]:
AUC9_6 = evaluator.evaluate(crime_m9_6_fitted.transform(validation_df))
AUC4_6 = evaluator.evaluate(crime_m4_6_fitted.transform(validation_df))

print("Model 4_6 - ALL + parameters AUC: ", AUC4_6)
print("Model 9_6 - category + time + parameters AUC: ", AUC9_6)

Model 4_6 - ALL + parameters AUC: 0.7652731332658433
Model 9_6 - category + time + parameters AUC: 0.7653104659499966

In [113]:
crime_m9_6_fitted.stages[-1].intercept

Out[ 94 ]: -0.4473666413923585

### Inference
- Feature importance for random forest and gradient boosting
- coefficients for logistic regression

In [114]:
m9_coe = crime_m9_fitted.stages[-1].coefficients.tolist()


In [115]:
# 4 level + 11 month + 9 location + 4 hour = 28
m9_feature = ['Minor', 'Medium', 'Servere',
                    'Very Servere','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug', 'Sep','Oct','Nov','SOUTHERN','MISSION','NORTHERN','CENTRAL',
                    'BAYVIEW','TENDERLOIN', 'INGLESIDE','TARAVAL','PARK','Near noon','Afternoon',
                    'evening','Night']


In [116]:
len(m9_coe)

Out[ 148 ]: 28

In [117]:
mapping = dict(zip(m9_feature, m9_coe))
mapping

Out[ 149 ]: 
{'Afternoon': 0.01199115462013647,
 'Apr': 0.0,
 'May': 0.0,
 'BAYVIEW': 0.49754645331235137,
 'SOUTHERN': 0.5014951815162885,
 'MISSION': 0.6524428839878921,
 'Medium': -1.3797986046888415,
 'Mar': 0.0,
 'Near noon': -0.22121125020509014,
 'TENDERLOIN': 1.3200871373838685,
 'Nov': 0.0,
 'Oct': 0.0,
 'Jan': 0.0,
 'Servere': -0.8160834161199783,
 'Jul': 0.0,
 'Feb': 0.0,
 'Sep': 0.0,
 'NORTHERN': 0.21790531801434798,
 'CENTRAL': 0.057285399872398474,
 'Jun': 0.0,
 'Very Servere': 1.2942751127905416,
 'INGLESIDE': 0.3034257893255847,
 'Aug': 0.0,
 'TARAVAL': 0.1512499835818672,
 'evening': -0.10139070895698639,
 'Minor': 0.5597343606159563,
 'PARK': 0.40445615084804204,
 'Night': -0.13146209171085055}

In [118]:
m9_feature_df = spark.createDataFrame(m9_feature, StringType()).toPandas()
m9_feature_df.head(3)
m9_feature_df.rename(columns={'value':'feature'},inplace=True)

In [119]:
#from pyspark.sql.types import IntegerType

# notice the parens after the type name
m9_coe_df = spark.createDataFrame(m9_coe, FloatType()).toPandas()

In [120]:
type(m9_coe_df)
m9_coe_df.rename(columns={'value':'coefficients'},inplace=True)

In [121]:
result = pd.concat([m9_feature_df, m9_coe_df], axis=1)
result['weights'] = (result['coefficients'] / 4)

In [122]:
result

Out[ 133 ]: 
 feature coefficients weights
0 Minor 0.226441 0.056610
1 Medium -0.633250 -0.158313
2 Servere -0.333160 -0.083290
3 Very Servere 0.380391 0.095098
4 Year -0.015564 -0.003891
5 Month -0.034911 -0.008728
6 SOUTHERN 0.193436 0.048359
7 MISSION 0.221585 0.055396
8 NORTHERN 0.066744 0.016686
9 CENTRAL 0.014685 0.003671
10 BAYVIEW 0.147883 0.036971
11 TENDERLOIN 0.396897 0.099224
12 INGLESIDE 0.084101 0.021025
13 TARAVAL 0.037344 0.009336
14 PARK 0.091782 0.022946
15 Near noon -0.086057 -0.021514
16 Afternoon 0.004879 0.001220
17 evening -0.037832 -0.009458
18 Night -0.053186 -0.013297

In [123]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

In [124]:
gb_m9_fit_model = gb_m9_fitted.stages[-1]
gb_m9_trans = gb_m9_fitted.transform(validation_df)

ExtractFeatureImp(gb_m9_fit_model.featureImportances, gb_m9_trans, "features").head(20)

Out[ 151 ]: 
 idx name score
1 1 Medium 0.198055
2 2 Servere 0.109800
24 24 Near noon 0.093585
16 16 MISSION 0.091373
0 0 Minor 0.063193
19 19 BAYVIEW 0.062820
18 18 CENTRAL 0.059466
20 20 TENDERLOIN 0.055668
22 22 TARAVAL 0.052691
3 3 Very Servere 0.048936
15 15 SOUTHERN 0.047045
26 26 evening 0.035096
27 27 Night 0.030860
25 25 Afternoon 0.020313
17 17 NORTHERN 0.017993
21 21 INGLESIDE 0.007367
23 23 PARK 0.005739
13 13 Oct 0.000000
12 12 Sep 0.000000
11 11 Aug 0.000000

In [125]:
rf_m2_fit_model = rf_m2_fitted.stages[-1]
m2_trans = rf_m2_fitted.transform(validation_df)

ExtractFeatureImp(rf_m2_fit_model.featureImportances, m2_trans, "features").head(20)

Out[ 152 ]: 
 idx name score
3 3 Very Servere 0.423025
0 0 Minor 0.265727
1 1 Medium 0.216403
2 2 Servere 0.052833
9 9 TENDERLOIN 0.028345
7 7 CENTRAL 0.006011
11 11 TARAVAL 0.004311
6 6 NORTHERN 0.002045
5 5 MISSION 0.001300
4 4 SOUTHERN 0.000000
8 8 BAYVIEW 0.000000
10 10 INGLESIDE 0.000000
12 12 PARK 0.000000